Pavan Kusampudi

700762366

### Q5: Evaluation Metrics from a Multi-Class Confusion Matrix

The system classified 90 animals into Cat, Dog, or Rabbit. The results are shown below:

System \ Gold	Cat	Dog	Rabbit

Cat	5	10	5

Dog	15	20	10

Rabbit	0	15	10

3.	Programming Implementation

Write Python code that:

1.	Accepts the confusion matrix above as input.
2.	Computes per-class precision and recall.
3.	Computes macro-averaged and micro-averaged precision and recall.
4.	Prints all results clearly.


In [4]:
import numpy as np
# Given Confusion matrix for 3 classes: Cat, Dog, Rabbit. Where rows are predicted labels and columns are true labels.
confusion_matrix = np.array([
    [5, 10, 5],   
    [15, 20, 10], 
    [0, 15, 10]   
])

classes = ["Cat", "Dog", "Rabbit"]  # Class labels

In [5]:

# 1. Computes per-class precision & recall
row_sums = confusion_matrix.sum(axis=1) 
col_sums = confusion_matrix.sum(axis=0) 

precisions = []
recalls = []

for i, label in enumerate(classes):
    TP = confusion_matrix[i, i]
    FP = row_sums[i] - TP
    FN = col_sums[i] - TP
    
    precision = TP / (TP + FP) if (TP + FP) > 0 else 0.0
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0.0
    
    precisions.append(precision)
    recalls.append(recall)
    
    print(f"{label}: Precision = {precision:.3f}, Recall = {recall:.3f}")

Cat: Precision = 0.250, Recall = 0.250
Dog: Precision = 0.444, Recall = 0.444
Rabbit: Precision = 0.400, Recall = 0.400


In [2]:
# 2. Computes macro-averaged precision & recall
macro_precision = np.mean(precisions)
macro_recall = np.mean(recalls)
print(f"\nMacro-averaged Precision = {macro_precision:.3f}")
print(f"Macro-averaged Recall = {macro_recall:.3f}")


Macro-averaged Precision = 0.365
Macro-averaged Recall = 0.365


In [3]:
# 3. Computes micro-averaged precision & recall
TP_total = np.trace(confusion_matrix)     
FP_total = row_sums.sum() - TP_total
FN_total = col_sums.sum() - TP_total

micro_precision = TP_total / (TP_total + FP_total)
micro_recall = TP_total / (TP_total + FN_total)

print(f"\nMicro-averaged Precision = {micro_precision:.3f}")
print(f"Micro-averaged Recall = {micro_recall:.3f}")


Micro-averaged Precision = 0.389
Micro-averaged Recall = 0.389


### Q8. Programming: Bigram Language Model Implementation (based on “Activity: I love NLP corpus” slide)

Tasks:

Write a Python program to:

1.	Read the training corpus:
2.	<s I love NLP /s>  
3.	<s I love deep learning /s>  
4.	<s deep learning is fun /s>
5.	Compute unigram and bigram counts.
6.	Estimate bigram probabilities using MLE.
7.	Implement a function that calculates the probability of any given sentence.
8.	Test your function on both sentences:
o	<s I love NLP /s>
o	<s I love deep learning </s>
9.	Print which sentence the model prefers and why.


In [6]:
from collections import Counter
from itertools import tee
import math

corpus = [
    ["<s>", "I", "love", "NLP", "</s>"],
    ["<s>", "I", "love", "deep", "learning", "</s>"],
    ["<s>", "deep", "learning", "is", "fun", "</s>"],
]
def bigrams(tokens):
    a, b = tee(tokens)
    next(b, None)
    return list(zip(a, b))

In [7]:
# Computes unigram and bigram counts
unigram_counts = Counter()
bigram_counts = Counter()
for sent in corpus:
    unigram_counts.update(sent)
    bigram_counts.update(bigrams(sent))

In [8]:
# Bigram MLE: P(w_i | w_{i-1}) = c(w_{i-1}, w_i) / c(w_{i-1})
def bigram_mle_prob(w_prev, w_cur):
    c_prev = unigram_counts[w_prev]
    c_bigram = bigram_counts.get((w_prev, w_cur), 0)
    if c_prev == 0:
        return 0.0
    return c_bigram / c_prev

In [9]:
# Optional: pretty table (dict of dicts)
cond_probs = {}
for (w_prev, w_cur), c in bigram_counts.items():
    cond_probs.setdefault(w_prev, {})[w_cur] = c / unigram_counts[w_prev]

# 4) Sentence probability (MLE bigram):

def sentence_prob(tokens):
    logp = 0.0
    for w_prev, w_cur in bigrams(tokens):
        p = bigram_mle_prob(w_prev, w_cur)
        if p == 0.0:
            return 0.0, float("-inf")
        logp += math.log(p)
    return math.exp(logp), logp


In [ ]:
# Testing  two sentences

s1 = ["<s>", "I", "love", "NLP", "</s>"]
s2 = ["<s>", "I", "love", "deep", "learning", "</s>"]

p1, lp1 = sentence_prob(s1)
p2, lp2 = sentence_prob(s2)

In [27]:
# 6) Printing results
print("------ Unigram Counts ------")
for tok, cnt in sorted(unigram_counts.items()):
    print(f"{tok:>10}: {cnt}")

------ Unigram Counts ------
      </s>: 3
       <s>: 3
         I: 2
       NLP: 1
      deep: 2
       fun: 1
        is: 1
  learning: 2
      love: 2


In [28]:
print("\n------ Bigram Counts ------")
for (w_prev, w_cur), cnt in sorted(bigram_counts.items()):
    print(f"{w_prev:>10}, {w_cur:<10} : {cnt}")


------ Bigram Counts ------
       <s>, I          : 2
       <s>, deep       : 1
         I, love       : 2
       NLP, </s>       : 1
      deep, learning   : 2
       fun, </s>       : 1
        is, fun        : 1
  learning, </s>       : 1
  learning, is         : 1
      love, NLP        : 1
      love, deep       : 1


In [29]:

print("\n------ Bigram Probabilities (MLE) ------")
for w_prev in sorted(cond_probs):
    for w_cur in sorted(cond_probs[w_prev]):
        print(f"P({w_cur}|{w_prev}) = {cond_probs[w_prev][w_cur]:.6f}")


------ Bigram Probabilities (MLE) ------
P(I|<s>) = 0.666667
P(deep|<s>) = 0.333333
P(love|I) = 1.000000
P(</s>|NLP) = 1.000000
P(learning|deep) = 1.000000
P(</s>|fun) = 1.000000
P(fun|is) = 1.000000
P(</s>|learning) = 0.500000
P(is|learning) = 0.500000
P(NLP|love) = 0.500000
P(deep|love) = 0.500000


In [30]:
print("\n------ Sentence Probabilities ------")
print(f"S1: {' '.join(s1)} -> P = {p1:.6f}, logP = {lp1:.6f}")
print(f"S2: {' '.join(s2)} -> P = {p2:.6f}, logP = {lp2:.6f}")


------ Sentence Probabilities ------
S1: <s> I love NLP </s> -> P = 0.333333, logP = -1.098612
S2: <s> I love deep learning </s> -> P = 0.166667, logP = -1.791759


In [26]:
preferred = "S1" if p1 > p2 else "S2" if p2 > p1 else "Tie"
print(f"\nPreferred Sentence from above: {preferred}.")
print(f"\n Reason: Here the Probability of S1 = 0.3333.Probability of S2 = 0.1667. Under the bigram MLE model, S1 has the higher probability because the sequence “love → NLP” followed by “NLP → </s>” matches the training data more strongly than “love → deep → learning → </s>”.")



Preferred Sentence from above: S1.

 Reason: Here the Probability of S1 = 0.3333.Probability of S2 = 0.1667. Under the bigram MLE model, S1 has the higher probability because the sequence “love → NLP” followed by “NLP → </s>” matches the training data more strongly than “love → deep → learning → </s>”.
